# Word2Vec 학습
## Tagging 된 CSV 가져오기
- 공통 변수 선언

In [1]:
import os 
import pandas as pd

PROJ_HOME = '\\'.join(os.getcwd().split('\\')[0:-1])
MARK_LST_CSV = f'{PROJ_HOME}\\crawled_files\\market_master.csv'
OKT_TAG_DIR = f'{PROJ_HOME}\\okt_tag_files'

In [27]:
stop_words = ['이','날','전','에','인데','엇','가','도','거','이랑','랑','은','너','요','면','할','것','의','있습니다','는','식']

In [26]:
all_tag_review_lst = []
for okt_tag_file in os.listdir(OKT_TAG_DIR):
    file = f'{OKT_TAG_DIR}\\{okt_tag_file}'
    review_df = pd.read_csv(file, encoding='utf-8')
    review_df = review_df.dropna(axis=0)
    for idx, row in review_df.iterrows():
        tagged_text = row['tagged_text']
        tagged_text_split_lst = tagged_text.split(' ')
        tagged_text_split_lst = [word for word in tagged_text_split_lst if word not in stop_words] 
        all_tag_review_lst.append(tagged_text_split_lst)

print(all_tag_review_lst[:3])

[['진짜', '맛', '써여', '여행', '마지막', '출발', '먹은', '음식', '너무', '맛있게', '잘', '먹', '숨다', '펩시', '제로', '카프리', '있다니', '그릇', '너무', '예', '쁘구', '너무', '맛', '잘알이신', '같아서', '감동', '였어여', '강릉', '여행', '꼭', '추천'], ['강릉', '동해', '쪽', '다니면서', '유명하다는', '햄버거', '집', '다', '먹어', '봤는데요', '여기', '제일', '맛있습니다', '퀄리티', '다르구요', '빵', '패티', '소스', '재료', '하나', '하나', '깨끗하고', '신선해요', '사장', '님', '의', '열정', '느껴지구요', '서비스', '좋으세요', '강릉', '오면', '무조건', '드시고', '가세', '초당', '버거', '타', '노스', '먹었', '는데요', '초당', '상큼', '한', '버거', '타', '노스', '는', '미국', '식', '미트볼', '소스', '같아요', '둘다', '특징', '확실하고', '매력', '있습니다', '웨', '지', '감자', '맛있', '갈릭'], ['강릉', '에서', '매번', '순두부', '장', '칼국수', '짬뽕', '만', '먹다가', '새로운', '음식', '시도', '해보려고', '수제', '버거', '검색', '해봤는데', '여기', '후기', '제일', '좋아서', '왔어요', '후기', '괜히', '좋', '게', '아니네요', '양도', '많고', '정말', '맛있어요', '특히', '패티', '장', '들어가있고', 'bbq', '소스', '들어가있는', '버거', '기억', '남습', '니당', '다음', '강릉', '여행', '또', '오게되', '여기', '재', '방문', '같아요']]


## Word2vec 학습시키기 
- 파라미터는 아래 참고 
    * vector_size: 차원의 크기, 100차원 120차원, 200차원 등 
    * window: 문장 내 중심 단어를 포함하여 양 옆으로 인접할 단어의 개수 
    * min_count: 단어의 최소 빈도수 제한 (이보다 작은 빈도의 단어는 미학습함)
    * workers: 학습을 위한 프로세스 개수 
    * sg: 0은 CBOW, 1은 SKIP-GRAM

In [28]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=all_tag_review_lst , vector_size = 100 , window = 5, min_count=3 , workers = 4 , sg = 0)

In [30]:
# 다른 값으로 바꿔가며 확인해보세요.
similar_rslt = model.wv.most_similar(positive=['짬뽕'], negative=['커피'])          # positive와는 유사하지만 negative와는 먼
print(similar_rslt)

[('붉은', 0.41380780935287476), ('b', 0.30172011256217957), ('a', 0.29738718271255493), ('순두부', 0.27546262741088867), ('물회', 0.23982565104961395), ('세트', 0.22132812440395355), ('회', 0.19141291081905365), ('편이었어요', 0.17840933799743652), ('간간', 0.17744366824626923), ('퀄', 0.15825864672660828)]


## 모델 저장, 모델 로드

In [14]:
model.wv.save_word2vec_format('./myw2vModel') #모델저장


In [15]:
# 모델 로드
#model.wv.load_word2vec_format('./myw2vModel')


## tsv 파일 생성
- projector.tensorflow.org 에 업로드용 tsv 파일 생성
- 두 개의 tsv 파일을 모두 projector.tensorflow.org 에 업로드하여 결과 확인 

In [21]:
TSV_DIR = f'{PROJ_HOME}\\tsv_files'
if not os.path.exists(TSV_DIR):
    os.mkdir(TSV_DIR)
    

with open(TSV_DIR + '\\tensors.tsv', 'w') as tensors:
    with open(TSV_DIR + '\\metadata.tsv', 'w') as metadata:
        for word in model.wv.index_to_key :
            metadata.write(word + '\n')
            vector_row = '\t'.join(map(str, model.wv[word]))
            tensors.write(vector_row + '\n')